In [47]:
import os

In [48]:
%pwd

'd:\\Weather_classification\\Weather-Image-Classification'

In [5]:
os.chdir("../")

In [6]:
%pwd

'd:\\Weather_classification\\Weather-Image-Classification'

In [49]:
from pathlib import Path
from dataclasses import dataclass
from typing import List

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir:Path
    tensorboard_root_log_dir:Path
    checkpoint_model_filepath:Path

In [50]:
from Weather_Image_Classification.constants import *
from Weather_Image_Classification.utils.common import read_yaml,create_directories

In [51]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(model_ckpt_dir),
                            Path(config.tensorboard_root_log_dir)
                            ])
        
        preapre_callbacks_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath),
        )
        
        return preapre_callbacks_config

In [52]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [53]:
class PrepareCallback:
    def __init__(self,config:PrepareCallbacksConfig):
        self.config = config
        
    @property
    def _create_tb_callback(self):
        timestamp = time.strftime("%Y-%m-%dT%H:%M:%S")
        
        tb_running_lo_dir = os.path.join(self.config.tensorboard_root_log_dir,f"tb_logs_at_{timestamp}")
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_lo_dir)
    

    @property
    def _create_ckpt_callback(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    
    def get_tb_ckpt_callback(self):
        
        return [
            self._create_tb_callback,
            self._create_ckpt_callback
        ]

In [54]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callbacks_list = prepare_callbacks.get_tb_ckpt_callback()
    
    
except Exception as e:
    raise e

[2024-11-28 15:52:51,577] || INFO || common || yaml file : config\config.yaml loaded successfully]
[2024-11-28 15:52:51,580] || INFO || common || yaml file : params.yaml loaded successfully]
[2024-11-28 15:52:51,581] || INFO || common || Directory artifacts already exists]
[2024-11-28 15:52:51,583] || INFO || common || Directory artifacts\prepare_callbacks\checkpoint_dir created successfully]
[2024-11-28 15:52:51,584] || INFO || common || Directory artifacts\prepare_callbacks\tensorboard_log_dir created successfully]


In [55]:
import time
timestamp = time.strftime("%Y-%m-%dT%H:%M:%S")
f"tb_logs_at_{timestamp}"

'tb_logs_at_2024-11-28T15:53:02'